This is the first notebook of the pipeline. The project is structured around three folders: Acquisition, Analysis and Code. 
Acquisition stores all the data, Analysis the results (and the intermediary results) and Code, the present notebooks. 
Both in Acquisition and Analysis we create one folder per experiment type (called batch). Inside each batch folder, we will keep a similar folder structure.

Create a batch name, a name that will encompass a hole set of experiments (which can have for instance different repetitions or / concentrations / time points).
Each batch will be stored in separate folder inside the analysis folder in which all the results will be stored.

Inside each batch, it it also possible to differentiate experiments by condition (for instance experiments vs control). 


23/05/25     Jacques Bourg @ Florian Muller lab. Institut Pasteur.


<img src="./HOX_pipeline.png" alt="LNP pipeline" width="1200" height="477">

In [2]:
import sys
from pathlib import Path
import skimage.io as io

import ipywidgets as widgets
from IPython.display import display

In [8]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.widgets import StringListWidget as Slw
from utils.file_handling import FileProcessor as Fp

fp  = Fp()
tk  = Track()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
text_input = widgets.Text(value='',placeholder='Batch name to input', description='', disabled=False)
display(text_input)

Text(value='', placeholder='Batch name to input')

In [6]:
batch_name = text_input.value; print(batch_name)

2024-11-18_Helena-Hox-9-10_spinning_2


In [7]:
folder_path_batch = Path(f"../Analysis/{batch_name}")
if not folder_path_batch.exists():
    folder_path_batch.mkdir(parents=True)
    
folder_path_batch = Path(f"../Acquisition/{batch_name}")
if not folder_path_batch.exists():
    folder_path_batch.mkdir(parents=True)

This line defines types of experiments/conditions that will take place in this batch: EXPERIMENT, CONTROL. We will create a single folder for each condition.
Set the names in capitals ! 

In [9]:
mods = Slw(); mods.display()

Text(value='', description='Input:', placeholder='Type something')

Button(description='Add to list', style=ButtonStyle(), tooltip='Click to add the input to the list')

Button(description='Delete list', style=ButtonStyle(), tooltip='Click to delete the list')

Output()

In [10]:
modalities = mods.string_list; print(modalities)

['EXPERIMENT', 'CONTROL']


In [11]:
for mod in modalities:
    folder_path_modality = Path(f"../Analysis/{batch_name}/{mod}")
    if not folder_path_modality.exists():
        folder_path_modality.mkdir(parents=True)
 
for mod in modalities:
    folder_path_modality = Path(f"../Acquisition/{batch_name}/{mod}")
    if not folder_path_modality.exists():
        folder_path_modality.mkdir(parents=True)

Now, lets defines the types of channels that appear in this experiment, for instance: DAPI and FISH (if there are several fish call them FISH_1, FISH_2 or some other name).
Set the names in capitals !

In [12]:
chans = Slw(); chans.display()

Text(value='', description='Input:', placeholder='Type something')

Button(description='Add to list', style=ButtonStyle(), tooltip='Click to add the input to the list')

Button(description='Delete list', style=ButtonStyle(), tooltip='Click to delete the list')

Output()

In [13]:
channels = chans.string_list; print(channels)

['DAPI', 'HOXC8', 'HOXA10']


Associate a color with each channel, ni the same order (write in lowercase or in hex), this will be useful when plotting the results. For instance DAPI is usually set to blue.

In [14]:
cols = Slw(); cols.display()

Text(value='', description='Input:', placeholder='Type something')

Button(description='Add to list', style=ButtonStyle(), tooltip='Click to add the input to the list')

Button(description='Delete list', style=ButtonStyle(), tooltip='Click to delete the list')

Output()

In [15]:
colors = cols.string_list; print(colors)

['blue', 'green', 'red']


Enter the structures we are going to segment (usually NUCLEI and CELL in uppercase).

In [16]:
structs = Slw(); structs.display()

Text(value='', description='Input:', placeholder='Type something')

Button(description='Add to list', style=ButtonStyle(), tooltip='Click to add the input to the list')

Button(description='Delete list', style=ButtonStyle(), tooltip='Click to delete the list')

Output()

In [17]:
strucs = structs.string_list; print(strucs)

['NUCLEI', 'CELL']


Create a folder for each channel inside each condition

In [18]:
for mod in modalities:
    for chan in channels:
        folder_path_mods_chans = Path(f"../Analysis/{batch_name}/{mod}/{chan}")
        if not folder_path_mods_chans.exists():
            folder_path_mods_chans.mkdir(parents=True)
            
for mod in modalities:
    for chan in channels:
        folder_path_mods_chans = Path(f"../Acquisition/{batch_name}/{mod}/{chan}")
        if not folder_path_mods_chans.exists():
            folder_path_mods_chans.mkdir(parents=True)

In [19]:
DAPI_CHANNEL   = 2   
GENE_0_CHANNEL = 0   
GENE_1_CHANNEL = 1

channels_num = [DAPI_CHANNEL, GENE_0_CHANNEL, GENE_1_CHANNEL]

In [20]:
modality     = 'EXPERIMENT'

list_files   = [Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 NT/NT6h/HM40-J3DIFF-NT6h-HOXA10-620_HOXC8-520-1002-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 NT/NT24h/HM40-J3DIFF-NT24h-HOXA10-620_HOXC8-520-1005-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 NT/NT48h/HM40-J3DIFF-NT48h-HOXA10-620_HOXC8-520-1008-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 GDF/GDF6h/HM40-J3DIFF-GDF6h-HOXA10-620_HOXC8-522-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 GDF/GDF24h/HM40-J3DIFF-GDF24h-HOXA10-620_HOXC8-525-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 GDF/GDF48h/HM40-J3DIFF-GDF48h-HOXA10-620_HOXC8-528-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 FGF GDF/FGFGDF6h/HM40-J3DIFF-FGFGDF6h-HOXA10-620_HOXC8-531-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 FGF GDF/FGFGDF24h/HM40-J3DIFF-FGFGDF24h-HOXA10-620_HOXC8-534-1.tif'),
                Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/WTC11 FGF GDF/FGFGDF48h/HM40-J3DIFF-FGFGDF48h-HOXA10-620_HOXC8-537-1.tif'),
                ]

for ind, chan in enumerate(channels_num):
    folder_dirs = []
    for ind_image, f_name in enumerate(list_files):                      # This cell separates the files in channels and stores them in the corresponding condition / channel folder
        im        = io.imread(f_name)
        im_c      = im[...,chan]                                         # careful ! this line often changes according to the file format 
        short_name= Path(f_name).stem
        file_path = str(Path(f"../Acquisition/{batch_name}/{modality}")/ Path(channels[ind]) / Path(short_name + f'_{channels[ind]}.tif'))
        io.imsave(file_path, im_c)
        folder_dirs.append(file_path)  
    exec(f"BATCH_{modality}_{channels[ind]} = folder_dirs", globals())

/tmp/ipykernel_477766/4063281243.py:21: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/HM40-J3DIFF-NT6h-HOXA10-620_HOXC8-520-1002-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/4063281243.py:21: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/HM40-J3DIFF-NT24h-HOXA10-620_HOXC8-520-1005-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/4063281243.py:21: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/HM40-J3DIFF-NT48h-HOXA10-620_HOXC8-520-1008-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/4063281243.py:21: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/HM40-J3DIFF-GDF6h-HOXA10-620_HOXC8-522-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/4063281243.py:21: UserWarning: ../Acquisition/2024-11-18_Helena-

In [22]:
modality    = 'CONTROL'

list_files  = [Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/2DiPSC/HM40-2DiPSC-HOXA10-620_HOXC8-522-1.tif'),
               Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/2DiPSC/HM40-2DiPSC-HOXA10-620_HOXC8-523-1.tif'),
               Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/2DiPSC/HM40-2DiPSC-HOXA10-620_HOXC8-524-1.tif'),
               Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/3DiPSC/HM40-3DiPSC-HOXA10-620_HOXC8-520-1004-1.tif'),
               Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/3DiPSC/HM40-3DiPSC-HOXA10-620_HOXC8-520-1005-1.tif'),     
               Path('/home/jacques/Documents/FISH/Data_analysis/pipeline_smfish/2024-11-18_Helena-Hox-9-10_spinning/Acquisition/03_2025/transfer_9458852_files_415095e1/1Z-FISH-CONTROL-HM40-HM17/3DiPSC/HM40-3DiPSC-HOXA10-620_HOXC8-520-1007-1.tif'),
            ] 
        
for ind, chan in enumerate(channels_num):
    folder_dirs = []
    for ind_image, f_name in enumerate(list_files):                  # This cell separates the files in channels and stores them in the corresponding condition / channel folder        
        
        im        = io.imread(f_name)
        im_c      = im[...,chan]
        short_name= Path(f_name).stem
        file_path = str(Path(f"../Acquisition/{batch_name}/{modality}")/ Path(channels[ind]) / Path(short_name + f'_{channels[ind]}.tif'))
        io.imsave(file_path, im_c)
        folder_dirs.append(file_path)
    exec(f"BATCH_{modality}_{channels[ind]} = folder_dirs", globals())

/tmp/ipykernel_477766/1133171386.py:19: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/CONTROL/DAPI/HM40-2DiPSC-HOXA10-620_HOXC8-522-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/1133171386.py:19: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/CONTROL/DAPI/HM40-2DiPSC-HOXA10-620_HOXC8-523-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/1133171386.py:19: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/CONTROL/DAPI/HM40-2DiPSC-HOXA10-620_HOXC8-524-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/1133171386.py:19: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/CONTROL/DAPI/HM40-3DiPSC-HOXA10-620_HOXC8-520-1004-1_DAPI.tif is a low contrast image
  io.imsave(file_path, im_c)
/tmp/ipykernel_477766/1133171386.py:19: UserWarning: ../Acquisition/2024-11-18_Helena-Hox-9-10_spinning_2/CONTROL/DAPI/HM40-3DiPSC-

As we are working on 2D images, we will copy them into the Analysis folder (/Analysis/{batch_name}/{modality}/{chan}/{struc}/2D_train)
in order to train the segmentation afterwards.

In [23]:
struc = 'NUCLEI'
 
for modality in modalities:
    
        if modality == 'EXPERIMENT':
            batch_mod_chan = BATCH_EXPERIMENT_DAPI
        elif modality == 'CONTROL':
            batch_mod_chan = BATCH_CONTROL_DAPI
    
        folder_struc          = Path(f"../Analysis/{batch_name}/{modality}/DAPI/NUCLEI/")
        if not folder_struc.exists():
            folder_struc.mkdir(parents=True)

        folder_mod_chan_mip = Path(f"../Analysis/{batch_name}/{modality}/DAPI/NUCLEI/train_2D")
        if not folder_mod_chan_mip.exists():
            folder_mod_chan_mip.mkdir(parents=True)

        batch_mod_chan_mip = []
        for file_path in batch_mod_chan: 
            file_path = Path(file_path)
            im = io.imread(file_path)
            new_file_name = '_'.join(Path(file_path).stem.split('_')[:-1]) + f'_MIP_DAPI_NUCLEI.tif'
            new_file_add  = str(folder_mod_chan_mip / Path(new_file_name))
            io.imsave(new_file_add, im, imagej=True)
            batch_mod_chan_mip.append(new_file_add)
        
        exec(f"BATCH_{modality}_DAPI_NUCLEI_MIP = batch_mod_chan_mip", globals())   ##### turn lower case variables into uppercase for tracking    

/tmp/ipykernel_477766/1735395383.py:24: UserWarning: ../Analysis/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/NUCLEI/train_2D/HM40-J3DIFF-NT6h-HOXA10-620_HOXC8-520-1002-1_MIP_DAPI_NUCLEI.tif is a low contrast image
  io.imsave(new_file_add, im, imagej=True)
/tmp/ipykernel_477766/1735395383.py:24: UserWarning: ../Analysis/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/NUCLEI/train_2D/HM40-J3DIFF-NT24h-HOXA10-620_HOXC8-520-1005-1_MIP_DAPI_NUCLEI.tif is a low contrast image
  io.imsave(new_file_add, im, imagej=True)
/tmp/ipykernel_477766/1735395383.py:24: UserWarning: ../Analysis/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/NUCLEI/train_2D/HM40-J3DIFF-NT48h-HOXA10-620_HOXC8-520-1008-1_MIP_DAPI_NUCLEI.tif is a low contrast image
  io.imsave(new_file_add, im, imagej=True)
/tmp/ipykernel_477766/1735395383.py:24: UserWarning: ../Analysis/2024-11-18_Helena-Hox-9-10_spinning_2/EXPERIMENT/DAPI/NUCLEI/train_2D/HM40-J3DIFF-GDF6h-HOXA10-620_HOXC8-522-1_MIP_DAPI_NUCLEI.tif i

Finally execute these last two cells to collect the constants.
The convention is to write the constants that we want to pass by to the other notebooks in uppercase at the end of the notebook.
This will create a json file that will be used to communicate between notebooks used in a same batch of experiments.

In [24]:
MODALITIES   = modalities
CHANNELS     = channels
BATCH_NAME   = batch_name
CHANNELS_NUM = channels_num
COLORS       = colors
STRUCTURES   = strucs

In [25]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, BATCH_NAME, folder_path = Path(f"../Analysis/{BATCH_NAME}"))

## END OF SCRIPT

This notebook has to be adapted to the specific file formats. We left some legacy code at the end of the notebook
containing examples of how to do this.


In [25]:
# path_folder = fp.select_file(initialdir=None)

for lif files:

In [ ]:
# images_names, image_list = fp.read_lif_many_images(path_folder) # since it is a lif file, for nd2 files see and adapt method read_convert_np

In [ ]:
# folder_path_batch = Path(f"../Analysis/{batch_name}/temp_file")
# if not folder_path_batch.exists():
#     folder_path_batch.mkdir(parents=True)

In [26]:
# path_names = []
# for ind, image in enumerate(image_list):
#     filename = Path(f"../Analysis/{batch_name}/temp_file") / Path(images_names[ind]+'.tif')
#     io.imsave(filename, image, imagej=True)
#     path_names.append(filename)

### Manual intervention required: change the channel numbers (for instance channel_dapi ) and verify that the channel corresponds to what you are looking for. 

In [27]:
# print(np.shape(image_list[0])) # dimensions of the images (#channels, # Z, #Y, #X)
# channel_dapi = 0               # verify which channels is which 
# #channel_fish = 1
# fig, ax = plt.subplots(figsize=(1,1))  
# ax.imshow(np.max(image_list[0][channel_dapi,...], axis = 0)) # for 3D images
# ax.set_axis_off();

In [28]:
# channel_dapi = 0     # this line is useful to separate the channels
# #channel_fish = 1
# print(channels)
# channels_num  = [channel_dapi]#, channel_fish]
# print('corresponding channels :',  channels_num)

##### For each condition: manually affect the experiments to each condition:
   Among the list of image names, choose the ones that must be affected to each condition

In [29]:
# print(images_names)
# list_same_cond = [images_names[0], images_names[1], images_names[3]]   # Choose the images for a given condition.
# indexes_ims    = [0, 1, 3]                                             # copy the indexes of the images 
# name_cond      = 'LNP'                                                 # the name of the condition/modalities must be among the modalities listed just before. 

In [30]:
# for ind_image, name in enumerate(list_same_cond):                      # This cell separates the files in channels and stores them in the corresponding condition / channel folder
#     folder_dirs = []
#     for ind, chan in enumerate(channels_num):
#         im_c      = image_list[indexes_ims[ind]][chan,...]
#         file_path = str(Path(f"../Acquisition/{batch_name}/{name_cond}")/ Path(channels[ind]) / Path(name + f'_{channels[ind]}.tif'))
#         io.imsave(file_path, im_c)
#         folder_dirs.append(file_path)
        
#     exec(f"BATCH_{name_cond}_{channels[ind]} = folder_dirs", globals())  # create the uppercased variables  (condition, channel) containing the list of all the associated files
#                                                                          # for instance BATCH_LNP_DAPI        

Once all the channels of each condition were affected in a specific folder as a separate .tif file, erase the /temp_file folder

In [31]:
# import shutil
# if os.path.exists(Path(f"../Analysis/{batch_name}/temp_file")):
#     shutil.rmtree(Path(f"../Analysis/{batch_name}/temp_file"))  # Remove the folder and all its contents

# B    
Provide directly the files names with the good names on them. If the files are already separated in conditions and in different channels (one tiff per channel).
Just reference them using the following synthax: BATCH_{condition}_{channel}, both conditions and channel in uppercase.

Warning : naming convention. We will assume in the rest of the pipeline, that the files between the name channels have similar names, excepting for the last suffix
which is for instance _DAPI or _FISH or _FISH0 or _FISH1 (you can decide on how to call them)

In [32]:
# channels_num   = [0, 1]

# BATCH_XPO1_FISH = [Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO-6Oligo/XPO-6Oligo_1_pos0_FISH.tif'),   
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO-6Oligo/XPO-6Oligo_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-12Oligo/XPO1-12Oligo_1_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-12Oligo/XPO1-12Oligo_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-12Oligo/XPO1-12Oligo_3_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-18Oligo/XPO1-18Oligo_1_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-18Oligo/XPO1-18Oligo_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-18Oligo/XPO1-18Oligo_3_pos0_FISH.tif'), 
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-24Oligo/XPO1-24Oligo_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-24Oligo/XPO1-24Oligo_3_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-24Oligo/XPO1-24Oligo_4_pos0_FISH.tif'),
#                   ]  

# BATCH_XPO1_DAPI = [Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO-6Oligo/XPO-6Oligo_1_pos0_DAPI.tif'),   
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO-6Oligo/XPO-6Oligo_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-12Oligo/XPO1-12Oligo_1_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-12Oligo/XPO1-12Oligo_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-12Oligo/XPO1-12Oligo_3_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-18Oligo/XPO1-18Oligo_1_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-18Oligo/XPO1-18Oligo_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-18Oligo/XPO1-18Oligo_3_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-24Oligo/XPO1-24Oligo_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-24Oligo/XPO1-24Oligo_3_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/XPO/XPO1-24Oligo/XPO1-24Oligo_4_pos0_DAPI.tif'),
#                   ]  

# for u in BATCH_XPO1_FISH:        # check those files exist:
#      print(os.path.isfile(u))     
# for u in BATCH_XPO1_DAPI:
#      print(os.path.isfile(u))
     
# BATCH_EP_FISH = [Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_1_pos0_FISH.tif'),  
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_2_pos0_FISH.tif'),  
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_3_pos0_FISH.tif'),
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_5_pos0_FISH.tif'),
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_6_pos0_FISH.tif'), ]

# BATCH_EP_DAPI = [Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_1_pos0_DAPI.tif'),  
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_2_pos0_DAPI.tif'),  
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_3_pos0_DAPI.tif'),
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_5_pos0_DAPI.tif'),
#                  Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/Electroporation/Electroporation-1/Electro-1_6_pos0_DAPI.tif'), ]

# for u in BATCH_EP_FISH:
#      print(os.path.isfile(u))
# for u in BATCH_EP_DAPI:
#      print(os.path.isfile(u))     
     
# BATCH_LNP_FISH = [Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_0/LNP_0_1_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_0/LNP_0_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_0/LNP_0_3_pos0_FISH.tif'),  
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_4/LNP_4_1_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_4/LNP_4_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_4/LNP_4_3_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_10/LNP_10_1_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_10/LNP_10_2_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_40/LNP_40_1_pos0_FISH.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_40/LNP_40_2_pos0_FISH.tif'),
#                   ]

# BATCH_LNP_DAPI = [Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_0/LNP_0_1_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_0/LNP_0_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_0/LNP_0_3_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_4/LNP_4_1_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_4/LNP_4_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_4/LNP_4_3_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_10/LNP_10_1_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_10/LNP_10_2_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_40/LNP_40_1_pos0_DAPI.tif'),
#                   Path('/media/PBI_big_data/rna-imaging/data/SANOFI-RNA-delivery/2023-05-11_LNP-SignalComparison/acquisition/LNP/LNP_40/LNP_40_2_pos0_DAPI.tif'),
#                   ]
 
# for u in  BATCH_LNP_FISH:
#      print(os.path.isfile(u))
     
# for u in BATCH_LNP_DAPI:
#      print(os.path.isfile(u))

Copy the folders locally to ../Acquisition/{batch_name}/{modality}/{channel}:

In [33]:
# for mod in modalities:    
#     for chan in channels:
#         file_path = eval(f'BATCH_{mod}_{chan}')
#         for file in file_path:
#             im_c = io.imread(file)
#             name = Path(file).name
#             file_path = str(Path(f"../Acquisition/{batch_name}/{mod}")/ Path(chan) / name)
#             io.imsave(file_path, im_c)

# End of A or B

#### Finally execute these last cells to collect the constants.
#### The convention is to write the constants that we want to pass by to the other notebooks in uppercase at the end of the notebook.
#### This will create a json file that will be used to communicate between notebooks used in a same batch of experiments.

In [ ]:
# MODALITIES   = modalities
# CHANNELS     = channels
# BATCH_NAME   = batch_name
# CHANNELS_NUM = channels_num
# COLORS       = colors
# STRUCTURES   = strucs

In [34]:
# constants2 = tk.collect_constants()
# tk.save_constants_and_commit_hash(constants2, BATCH_NAME, folder_path = Path(f"../Analysis/{BATCH_NAME}"))